In [80]:
# multivariate multi-step stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
import optuna
import pandas as pd
from empresa4.datasets import get_dataset

In [81]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out+1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix-2, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [100]:
df_orig = get_dataset('02_productos_todos')
df_orig = df_orig[(df_orig["product_id"].isin([20001, 20002])) & (df_orig["periodo"] == 201904)]
df_orig

{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()


,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados
22888,201904,20001,478.0,1757.73271,1647.63848,HC,ROPA LAVADO,3000.0,0
22889,201904,20002,454.0,1360.44402,1287.62346,HC,ROPA LAVADO,3000.0,0


In [102]:
df_orig = get_dataset('02_productos_todos_anti_leak')
df_orig = df_orig[df_orig['product_id'].isin([20001, 20002])]
df_orig.sort_values(by=['product_id', 'periodo'], inplace=True)
df_orig.reset_index(inplace=True, drop=True)    
df_orig

{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()


,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0
1,201702,20001,432.0,833.72187,798.01620,HC,ROPA LAVADO,3000.0,0
2,201703,20001,509.0,1330.74697,1303.35771,HC,ROPA LAVADO,3000.0,0
3,201704,20001,279.0,1132.94430,1069.96130,HC,ROPA LAVADO,3000.0,0
4,201705,20001,701.0,1550.68936,1502.20132,HC,ROPA LAVADO,3000.0,0
5,201706,20001,570.0,1575.82891,1520.06539,HC,ROPA LAVADO,3000.0,0
6,201707,20001,381.0,1086.47101,1030.67391,HC,ROPA LAVADO,3000.0,0
7,201708,20001,643.0,1289.66869,1267.39462,HC,ROPA LAVADO,3000.0,0
8,201709,20001,381.0,1356.96103,1316.94604,HC,ROPA LAVADO,3000.0,0
9,201710,20001,273.0,1441.60247,1439.75563,HC,ROPA LAVADO,3000.0,0


In [103]:
# define input sequence
in_seq1 = array(df_orig["cust_request_qty"])
in_seq2 = array(df_orig["cust_request_tn"])
in_seq3 = array(df_orig["tn"])
out_seq = array([df_orig.loc[i+2, "tn"] if i < len(df_orig) - 2 else 0 for i in range(len(df_orig))])

In [104]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [105]:
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, in_seq3, out_seq))
dataset



array([[ 479.     ,  937.72717,  934.77222, 1303.35771],
       [ 432.     ,  833.72187,  798.0162 , 1069.9613 ],
       [ 509.     , 1330.74697, 1303.35771, 1502.20132],
       [ 279.     , 1132.9443 , 1069.9613 , 1520.06539],
       [ 701.     , 1550.68936, 1502.20132, 1030.67391],
       [ 570.     , 1575.82891, 1520.06539, 1267.39462],
       [ 381.     , 1086.47101, 1030.67391, 1316.94604],
       [ 643.     , 1289.66869, 1267.39462, 1439.75563],
       [ 381.     , 1356.96103, 1316.94604, 1580.47401],
       [ 273.     , 1441.60247, 1439.75563, 1049.3886 ],
       [ 519.     , 1586.94356, 1580.47401, 1169.07532],
       [ 435.     , 1098.83929, 1049.3886 , 1043.7647 ],
       [ 437.     , 1256.01136, 1169.07532, 1856.83534],
       [ 302.     , 1150.37849, 1043.7647 , 1251.28462],
       [ 591.     , 1902.79056, 1856.83534, 1293.89788],
       [ 384.     , 1286.12277, 1251.28462, 1150.79169],
       [ 456.     , 1303.62129, 1293.89788, 1470.41009],
       [ 497.     , 1198.50145,

In [87]:
X, y = split_sequences(dataset, 3, 1)
X

array([[[ 479.     ,  937.72717,  934.77222],
        [ 432.     ,  833.72187,  798.0162 ],
        [ 509.     , 1330.74697, 1303.35771]],

       [[ 432.     ,  833.72187,  798.0162 ],
        [ 509.     , 1330.74697, 1303.35771],
        [ 279.     , 1132.9443 , 1069.9613 ]],

       [[ 509.     , 1330.74697, 1303.35771],
        [ 279.     , 1132.9443 , 1069.9613 ],
        [ 701.     , 1550.68936, 1502.20132]],

       [[ 279.     , 1132.9443 , 1069.9613 ],
        [ 701.     , 1550.68936, 1502.20132],
        [ 570.     , 1575.82891, 1520.06539]],

       [[ 701.     , 1550.68936, 1502.20132],
        [ 570.     , 1575.82891, 1520.06539],
        [ 381.     , 1086.47101, 1030.67391]],

       [[ 570.     , 1575.82891, 1520.06539],
        [ 381.     , 1086.47101, 1030.67391],
        [ 643.     , 1289.66869, 1267.39462]],

       [[ 381.     , 1086.47101, 1030.67391],
        [ 643.     , 1289.66869, 1267.39462],
        [ 381.     , 1356.96103, 1316.94604]],

       [[ 643.     ,

In [88]:
y

array([[1502.20132],
       [1520.06539],
       [1030.67391],
       [1267.39462],
       [1316.94604],
       [1439.75563],
       [1580.47401],
       [1049.3886 ],
       [1169.07532],
       [1043.7647 ],
       [1856.83534],
       [1251.28462],
       [1293.89788],
       [1150.79169],
       [1470.41009],
       [1800.96168],
       [1438.67455],
       [2295.19832],
       [1813.01511],
       [1486.68669],
       [1275.77351],
       [1259.09363]])

In [89]:
x_input = dataset[-3:, :-1]
x_input

array([[ 453.     , 1562.28968, 1486.68669],
       [ 370.     , 1371.7643 , 1275.77351],
       [ 367.     , 1343.99435, 1259.09363]])

# BO

In [90]:
def train_model(n_layers, n_neurons, dropout_threshold, n_steps, n_steps_out, X, y, embudo, embudo_2, embudo_3):
    # Train a single model where product_id is part of the input
    print(f"Training model with {n_layers} layers, {n_neurons} neurons, {n_steps} steps, {dropout_threshold} dropout, {embudo} embudo, {embudo_2} embudo_2 y {embudo_3} embudo_3.")

    # Number of features 
    n_features = X.shape[2]

    # Define the LSTM model
    if n_layers == 1:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", input_shape=(n_steps, n_features)))
        model.add(Dense(n_steps_out))
        model.compile(optimizer="adam", loss="mse")
    elif n_layers == 2:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo), activation="relu"))
        model.add(Dense(n_steps_out))
        model.compile(optimizer="adam", loss="mse")
    elif n_layers == 3:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo), activation="relu", return_sequences=True))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo_2), activation="relu"))
        model.add(Dense(n_steps_out))
        model.compile(optimizer="adam", loss="mse")
    elif n_layers == 4:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo), activation="relu", return_sequences=True))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo_2), activation="relu", return_sequences=True))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(int(n_neurons // embudo_3), activation="relu"))
        model.add(Dense(n_steps_out))
        model.compile(optimizer="adam", loss="mse")
    else:
        raise ValueError("n_layers must be 1, 2, 3 or 4")


    # Fit the model
    model.fit(X, y, epochs=200, verbose=0)
    return model

In [91]:
def make_predictions(n_steps_out, model, X_input):

    # Make prediction
    yhat = model.predict(X_input, verbose=1)

    values = []
    for i in range(n_steps_out):
        values.append(yhat[0][i])

    print(values)
    print(yhat)
    return values

In [92]:
def calculate_score(predictions, actuals):
    error = 0
    real_sum = sum(actuals)
    for i in range(len(predictions)):
        error += abs(predictions[i] - actuals[i])
    return error / real_sum

In [93]:
def objective(trial):
    # Define the search space using Optuna
    n_layers = trial.suggest_int('n_layers', 1, 4)
    n_neurons = trial.suggest_int('n_neurons', 20, 1000)
    n_steps_in = trial.suggest_int('n_steps', 1, 5)
    embudo = trial.suggest_float('embudo', 1, 2.5)
    embudo_2 = trial.suggest_float('embudo_2', 1, 3.5)
    embudo_3 = trial.suggest_float('embudo_3', 1, 4.5)
    dropout_threshold = trial.suggest_float('dropout_threshold', 0.1, 0.66)

    n_steps_out = 1

    # Your existing logic to train and evaluate the model
    X, y = split_sequences(dataset, n_steps_in, n_steps_out)

    x_input = dataset[-n_steps_in:, :-1]
    n_features = x_input.shape[1]

    
    # demonstrate prediction
    x_input = x_input.reshape((1, n_steps_in, n_features))

    # X_test, y_test = split_sequences(test_df.values, n_steps)
    # X_predict, y_predict = split_sequences(predict_df.values, n_steps)

    model = train_model(n_layers, n_neurons, dropout_threshold, n_steps_in, n_steps_out, X, y, embudo, embudo_2, embudo_3)
    predictions = make_predictions(n_steps_out, model, x_input)
    actuals = [1647, 1287]  # [1647, 1287]
    score = calculate_score(predictions, actuals)
    print(f"Score: {score}")
    return score

In [94]:
# Create a study object and specify the optimization direction
study = optuna.create_study(direction='minimize', study_name="lstm_8_BO_test")

# Perform optimization
study.optimize(objective, n_trials=50)

# Extract the best trial
best_trial = study.best_trial

print('Best trial:')
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

[I 2023-11-25 13:34:45,911] A new study created in memory with name: lstm_8_BO_test


{'out': None}
(1, 4)
{'out': None}
(20, 1000)
{'out': None}
(1, 5)
Training model with 1 layers, 106 neurons, 5 steps, 0.47269857131766657 dropout, 1.7233929338351173 embudo, 1.0773193830253698 embudo_2 y 1.2014509083298164 embudo_3.


1/1 [==============================] - 0s 82ms/step


[I 2023-11-25 13:34:47,015] Trial 0 finished with value: 0.037509679649552215 and parameters: {'n_layers': 1, 'n_neurons': 106, 'n_steps': 5, 'embudo': 1.7233929338351173, 'embudo_2': 1.0773193830253698, 'embudo_3': 1.2014509083298164, 'dropout_threshold': 0.47269857131766657}. Best is trial 0 with value: 0.037509679649552215.


[1708.7784]
[[1708.7784]]
Score: 0.037509679649552215
{'out': None}
(1, 4)
{'out': None}
(20, 1000)
{'out': None}
(1, 5)
Training model with 3 layers, 249 neurons, 5 steps, 0.5520174460951357 dropout, 2.0871977231034804 embudo, 2.1114060083348014 embudo_2 y 1.844023065455886 embudo_3.
1/1 [==============================] - 0s 164ms/step


...

[1185.0929]
[[1185.0929]]
Score: 0.2804536153565194
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.5714285714285714, 4.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.08888888888888889, 4.0)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'axis': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(140.14285714285714, 981.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(21.8, 981.0)
{'decimals': 0, 'out': None}
()
{'out': None}
(20, 1000)
{'decimals': 0, 'out': None}
()
{'out': None}
(20, 1000)
{'axis': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.7142857142857143, 5.0)
{'axis': -1, 'kind': None

[I 2023-11-25 13:36:54,823] Trial 48 finished with value: 0.030634534380692167 and parameters: {'n_layers': 2, 'n_neurons': 362, 'n_steps': 5, 'embudo': 1.9601352844940174, 'embudo_2': 1.5109173533957097, 'embudo_3': 2.0919969776704748, 'dropout_threshold': 0.437625236795162}. Best is trial 23 with value: 0.010205211535177595.


[1697.4551]
[[1697.4551]]
Score: 0.030634534380692167
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.5714285714285714, 4.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.08695652173913043, 4.0)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'decimals': 0, 'out': None}
()
{'out': None}
(1, 4)
{'axis': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(140.14285714285714, 981.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(21.32608695652174, 981.0)
{'decimals': 0, 'out': None}
()
{'out': None}
(20, 1000)
{'decimals': 0, 'out': None}
()
{'out': None}
(20, 1000)
{'axis': None, 'out': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.7142857142857143, 5.0)
{'axis': -

[I 2023-11-25 13:36:57,111] Trial 49 finished with value: 0.10956192248216454 and parameters: {'n_layers': 2, 'n_neurons': 365, 'n_steps': 4, 'embudo': 2.0097669522064794, 'embudo_2': 1.3168199299059529, 'embudo_3': 2.3977094678351167, 'dropout_threshold': 0.4356841272415419}. Best is trial 23 with value: 0.010205211535177595.


[1466.5515]
[[1466.5515]]
Score: 0.10956192248216454
Best trial:
n_layers: 1
n_neurons: 463
n_steps: 4
embudo: 1.5439480083368355
embudo_2: 1.7615661579465858
embudo_3: 1.9855036651574696
dropout_threshold: 0.4806804725288995
